In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from sklearn.cluster import KMeans
from scipy.ndimage import binary_dilation
from matplotlib.colors import ListedColormap
import matplotlib.colors as mcolors
from rasterio.errors import RasterioIOError
import csv
from skimage import io
import requests
from PIL import Image
from io import BytesIO

In [ ]:
tile = '11SKU'
location = 'Isla_vista_kelp'
cloud_cover_threshold = .25
save_mask = True
save_classification = True
path = os.path.join(r'H:\HLS_data\imagery',location,tile)
#path = r'C:\Users\attic\HLS_Kelp\imagery\Isla_vista_kelp_2018\11SKU'
item = 'HLS.S30.T11SKU.2017297T184441.v2.0'

#img_path = r'C:\Users\attic\HLS_Kelp\imagery\Isla_Vista_Kelp\10SGD\HLS.L30.T10SGD.2017167T183413.v2.0'

In [ ]:
img_path = os.path.join(path,item)
files = os.listdir(img_path)
file_data = item.split('.')
sensor = file_data[1]
if(sensor == 'L30'):
    sensor_bands = ['B02','B03','B04','B05','B06','B07'] #2,3,4,5,6,7]
else:
    sensor_bands = ['B02','B03','B04','B8A','B11','B12']
pattern = re.compile(r'\.(' + '|'.join(sensor_bands) + r')\.tif$')
img_files = [f for f in files if re.search(pattern, f)]
def get_band_index(filename):
    match = re.search(r'\.(B\d{2}|B8A)\.tif$', filename)
    if match:
        return sensor_bands.index(match.group(1))
    return -1

img_files = sorted(img_files, key=get_band_index)

geotiff_path = os.path.join(img_path, img_files[0])

with rasterio.open(geotiff_path) as dst:   
    hls = dst.read()
    dem_path = r'C:\Users\attic\HLS_Kelp\imagery\Socal_DEM.tiff'
    dem = rasterio.open(dem_path)
    if (dem.crs != dst.crs):
        reprojected_dem = np.zeros((hls.shape[1], hls.shape[2]), dtype=hls.dtype)
        reproject(
            source=dem.read(),
            destination=reprojected_dem,
            src_transform=dem.transform,
            src_crs=dem.crs,
            dst_transform=dst.transform,
            dst_crs=dst.crs,
            resampling=Resampling.bilinear)
    hls_flat = np.squeeze(hls, axis=0)   

if reprojected_dem.any():
    struct = np.ones((5,5))
    land_mask = binary_dilation(reprojected_dem > 0, structure = struct)
    ocean_mask = binary_dilation(reprojected_dem < -60 , structure = struct)
    full_mask = land_mask + ocean_mask
    # plt.figure(figsize=(6, 6))
    # plt.imshow(land_mask, cmap='gray')
    # plt.show()    
    # plt.figure(figsize=(6, 6))
    # plt.imshow(full_mask, cmap='gray')
    # plt.show()
    if save_mask:
        mask_path = os.path.join(path,f'{tile}_fullmask.tif')
        transform = dst.transform  
        height, width = full_mask.shape
        profile = {
            'driver': 'GTiff',
            'width': width,
            'height': height,
            'count': 1,  # one band
            'dtype': rasterio.uint8,  # assuming binary mask, adjust dtype if needed
            'crs': dst.crs,
            'transform': transform,
            'nodata': 0  # assuming no data is 0
        }

        # Write the land mask array to GeoTIFF
        with rasterio.open(mask_path, 'w', **profile) as dst:
            dst.write(full_mask.astype(rasterio.uint8), 1)
else:
    print("Something failed, you better go check...")

In [ ]:

 ##==========Select Granule and Get File Names==========##

files = os.listdir(img_path)
img_files = [f for f in files if re.search(pattern, f)]

def get_band_index(filename):
    match = re.search(r'\.(B\d{2}|B8A)\.tif$', filename)
    if match:
        return sensor_bands.index(match.group(1))
    return -1

img_files = sorted(img_files, key=get_band_index)

f_mask = [f for f in files if re.search(r'Fmask\.tif$', f)]

if not len(img_files)  == 6:
    print(f"incomplete file download: {item}")
img_bands = []
metadata_file = [f for f in files if re.search(r'metadata\.csv$', f)]
if metadata_file :
    with open(os.path.join(path,item, metadata_file[0]), mode='r') as file:
        csv_reader = csv.reader(file)
        keys = next(csv_reader)  
        values = next(csv_reader) 
    metadata = dict(zip(keys, values))

##==========Fmask Cloud mask==========##
#bitwise operations are weird. Far outside my comfort zone. Need to take CS33 first.........
with rasterio.open(os.path.join(img_path,f_mask[0])) as fmask:
    qa_band = fmask.read(1)
qa_bit = (1 << 1) - 1
qa_cloud_mask = ((qa_band >> 1) & qa_bit) == 1  # Bit 1 for cloud
qa_adjacent_to_cloud_mask = ((qa_band >> 2) & qa_bit) == 1  # Bit 2 for cloud adjacent
qa_cloud_shadow = ((qa_band >> 3) & qa_bit) == 1 
qa_ice = ((qa_band >> 4) & qa_bit) == 1 
#qa_water = ((qa_band >> 5) & qa_bit) == 1
qa_aerosol = (((qa_band >> 6) & 1) == 1) & (((qa_band >> 7) & 1) == 1)
cloud_mask = qa_cloud_mask #| qa_ice | qa_aerosol#Mask out Clouds and cloud-adjacent pixels  qa_cloud_mask #| qa_cloud_shadow | 
cloud_mask_2D = cloud_mask.reshape(-1).T
#may not be necessary to mask out the cloud-adjacent pixels 
#print(img_2D_normalized.shape)
#print(cloud_mask_2D.shape)

##========== Determine percentage of ocean covered by clouds ==========##
cloud_land_mask = cloud_mask | land_mask
cloud_but_not_land_mask = cloud_mask & ~land_mask
num_pixels_cloud_not_land = np.count_nonzero(cloud_but_not_land_mask)
num_pixels_not_land = np.count_nonzero(land_mask)
percent_cloud_covered = num_pixels_cloud_not_land/num_pixels_not_land
print(f'Percent cloud covered: {percent_cloud_covered}')

##==========Create stacked np array, Apply landmask==========##
print(img_files)
try:
    for file in img_files:
        with rasterio.open(os.path.join(img_path, file)) as src:
            img_bands.append(np.where(cloud_land_mask, 0, src.read(1)))  # Create image with the various bands
except RasterioIOError as e:
    print(f"Error reading file {file} in granule {item}: {e}")

img = np.stack(img_bands, axis=0)
n_bands, height, width = img.shape
img_2D = img.reshape(img.shape[0], -1).T #classifier takes 2D array of band values for each pixel 

##========== Normalize multi-spectral data ==========##
img_sum = img_2D.sum(axis=1)
epsilon = 1e-10  

img_2D_normalized = np.divide(img_2D, img_sum[:, None] + epsilon, where=(img_sum[:, None] != 0))
img_2D_normalized = (img_2D_normalized * 255).astype(np.uint8)
#normalized_img_bands = np.column_stack((img_2D_normalized, cloud_mask_2D))

In [ ]:
n_clusters_1 = 25#set this to determine number of clusters
kmeans = KMeans(n_clusters=n_clusters_1, random_state=42)
clusters_1 = kmeans.fit_predict(img_2D_normalized)
kmeans_1 = clusters_1.reshape((height, width))

In [ ]:
cloud_mask_1 = np.zeros_like(kmeans_1)
clustering_1 = np.copy(kmeans_1)
clouds_land_1 = [0]#[1,2,3,4,5,7,8,9]

kelp_1 = []
ocean_1 = range(1,25)

cloud_land_mask_1 = np.isin(kmeans_1, clouds_land_1)
clustering_1[cloud_land_mask_1] = 2
kelp_mask_1 = np.isin(kmeans_1, kelp_1)
clustering_1[kelp_mask_1] = 0
ocean_mask_1 = np.isin(kmeans_1,ocean_1)
clustering_1[ocean_mask_1] = 1
clustering_1_cropped = clustering_1[2800:3200,800:1600]
kmeans_1_cropped = kmeans_1[2800:3200,800:1600]
plt.figure(figsize=(25, 25))
plt.imshow(kmeans_1_cropped,  alpha=.25)
plt.imshow(clustering_1_cropped, alpha=.75)
plt.title('clustered')
plt.colorbar()


In [ ]:
r_nor = img_2D[:,2].reshape((height, width))
g_nor = img_2D[:,1].reshape((height, width))
b_nor = img_2D[:,0].reshape((height, width))
rgb_nor = np.stack([r_nor,g_nor,b_nor], axis=-1)
#rgb_nor_cropped = rgb_nor#[2500:3400,600:2000]

rgb_cropped = rgb_nor[2800:3050,850:1600]
plt.figure(figsize=(25, 25))
plt.subplot(2, 1, 2) 
plt.imshow(rgb_cropped)
plt.title("RGB Cropped Image")
plt.show()

In [ ]:
empty_band = np.ones(len(img_2D[0,:]))*1000
kmeans_1_2D = clustering_1.reshape(-1).T

kmeans_1_landmask = np.copy(img_2D)
for i, ocean_pixel in enumerate(kmeans_1_2D):
    if not ocean_pixel == 0:
        kmeans_1_landmask[i,:] = empty_band

In [ ]:
n_clusters_2 = 10
kmeans = KMeans(n_clusters=n_clusters_2, random_state=42)
clusters_2 = kmeans.fit_predict(kmeans_1_landmask)
kmeans_2 = clusters_2.reshape((height, width))

In [ ]:
# 0 = kelp, 1 = ocean, 2 = misc (land,clouds)
cloud_mask_2 = np.zeros_like(kmeans_2)
kelp_mask_2 = np.zeros_like(kmeans_2)
misc_mask_2 = np.zeros_like(kmeans_2)

clustering_2 = np.copy(kmeans_2)
ocean_2 = []#[1,2,3,4,5,7,8,9]
misc_2 = [0]
kelp_2 = []
kelp_mask_2 = np.isin(kmeans_2, kelp_2)
clustering_2[kelp_mask_2] = 0
ocean_mask_2 = np.isin(kmeans_2, ocean_2)
clustering_2[ocean_mask_2] = 1
misc_mask_2 = np.isin(kmeans_2,misc_2)
clustering_2[misc_mask_2] = 2

clustering_2_cropped = clustering_2[2700:3400, 800:1600]#[2700:3400, 600:2000] #[2500:3600,0:3600]
kmeans_2_cropped = kmeans_2#[2700:3400, 600:2000]#[2700:3400, 600:2000][2500:3600,0:36000]
colors = ['red', 'green', 'blue']
cmap_custom = ListedColormap(colors)


plt.figure(figsize=(20, 10))
#plt.imshow(kmeans_2_cropped, cmap='tab10', alpha=0)
plt.imshow(clustering_2_cropped, alpha=.75)
plt.title('clustered')
# Plotting the HLS data in red
#plt.imshow(img_cropped, cmap='grey', alpha=.5)
plt.colorbar()
plt.show()

In [ ]:
if False:
    empty_band = np.zeros(len(img_2D[0,:]))
    kmeans_1_2D = clustering_2.reshape(-1).T

    kmeans_1_landmask = np.copy(img_2D)
    for i, ocean_pixel in enumerate(kmeans_1_2D):
        if not ocean_pixel == 0:
            kmeans_1_landmask[i,:] = empty_band

In [ ]:
ocean_mask = (clustering_2 == 1) | (clustering_1 == 1)
kelp_mask = (clustering_2 == 0) 
#cloud_but_not_land_mask #This is a reminder to use this pre-made mask 

combined_mask = np.full(kelp_mask.shape, 1)
combined_mask[land_mask == 1] = 3
combined_mask[cloud_but_not_land_mask == 1] = 2
combined_mask[ocean_mask == 1] = 1
combined_mask[kelp_mask == 1] = 0

plt.figure(figsize=(20, 10))
plt.imshow(combined_mask, cmap='Blues')
plt.show()

#print(combined_mask[0:20,0:20])

In [ ]:
img_bands_2 =[]
try:
    for file in img_files:
        with rasterio.open(os.path.join(img_path, file)) as src:
            img_bands_2.append(np.where(land_mask, 0, src.read(1)))  # Create image with the various bands
except RasterioIOError as e:
    print(f"Error reading file {file} in granule {item}: {e}")

img = np.stack(img_bands_2, axis=0)
n_bands, height, width = img.shape
img_2D = img.reshape(img.shape[0], -1).T #classifier takes 2D array of band values for each pixel 
img_sum = img_2D.sum(axis=1)
epsilon = 1e-10  

img_2D_normalized = np.divide(img_2D, img_sum[:, None] + epsilon, where=(img_sum[:, None] != 0))
img_2D_normalized = (img_2D_normalized * 255).astype(np.uint8)



In [ ]:

# with rasterio.open(r"C:\Users\attic\HLS_Kelp\imagery\rf_training_v5\HLS.L30.T11SKU.2018042T183356.v2.0_kelp_classified.tif") as classified:
#            new_classified = classified.read(7)  # Create image with the various bands

In [ ]:
if save_classification:
    if not os.path.isdir (r'C:\Users\attic\HLS_Kelp\imagery\rf_training_unclassified'):
        os.mkdir(r'C:\Users\attic\HLS_Kelp\imagery\rf_training_unclassified')
    classification_path = os.path.join(r'C:\Users\attic\HLS_Kelp\imagery\rf_training_unclassified',f'{item}_kelp.tif')
    transform = dst.transform  
    height, width = clustering_2.shape
    reshaped_normalized_img = img_2D_normalized.reshape(height, width, 6)
    combined_img = np.dstack((reshaped_normalized_img,combined_mask))

    plt.figure(figsize=(6, 6))
    plt.imshow(combined_img[:,:,6], cmap='gray')
    plt.show()  

    num_bands = 7
    data_type = rasterio.uint8
    profile = {
        'driver': 'GTiff',
        'width': width,
        'height': height,
        'count': 7,  # one band
        'dtype': data_type,  # assuming binary mask, adjust dtype if needed
        'crs': dst.crs,
        'transform': transform,
        'nodata': 0  # assuming no data is 0
    }
    # Write the land mask array to GeoTIFF
    with rasterio.open(classification_path, 'w', **profile) as dst:
        for i in range(num_bands):
            dst.write(combined_img[:,:,i].astype(data_type), i + 1)